In [1]:
import pandas as pd, numpy as np, datetime as dt
import bayes_net_utils as bn
import matplotlib.pyplot as plt

# Introduction


# Read daily observed lake data and resample to seasonal frequency

If the forecast is being issued in April, then water chemistry and ecology data from the previous summer needs updating to include values from the previous year. Involves updating files two files. Probably best to just ask Sigrid Haande for this data and do it manually once a year:

cyanobacterial count data: '../../Data/Observed_Chem_Ecol/Van2_Vanemfjorden_Cyanobacteria.csv'
lake TP, chla and colour data: '../../Data/Observed_Chem_Ecol/Van2_Vanemfjorden_chem_obs.csv'

# Define and re-fit the BBN
   
Based on notebook 04_MakeHistoricTrainingData onwards, to create a new bn.fit object.

For the forecast issued in April, the seasonal weather forecast for the next three months (May-July) is accompanied by a forecast of water chemistry/ecology for the whole growing season (May-October). The forecast is for mean TP, chla, and colour concentration, and maximum cyanobacteria biovolume. This period and these variables match those used in WFD assessment of ecological status here.

All forecasts were originally going to be based on a Bayesian Belief Network (BBN) which included several weather-related nodes (mean seasonal wind speed and seasonal precipitation sum). However, the results of cross validation of the Bayesian Network and different versions of the ntework (notebook BN_CV_PythonPostProcess), and a comparison of different models for the hindcast period (notebook Hindcast_stats_and_plots), lead to the following choices for models to use in operational forecasting:

- TP: BBN (no met, as even in the BBN with met nodes, they don't affect the TP node anyway)
- chla: Naive seasonal forecast
- colour: BBN, no met (stats were the same for BBN with met, without met, or seasonal naive. Choose this for consistency with cyano)
- cyano: BBN, no met

# Re-generate predictions for the historic period and update GoF stats on those predictions

# Re-do the time series plots of predicted and observed for the historic period (Notebook 08_Hindcast_stats_and_plots), which are linked to in the tool

# Lake chemistry and ecology forecast for the summer season

**Intro**



For those nodes where the prediction will be based on the BBN without met data drivers, the forecast is still derived using the BBN which includes met nodes. However, the met nodes are not used when setting evidence for forecasting. This appears to give the same result as removing the nodes from the network altogether.

**Predictions for the past or the future?**
The models use measured lake chemistry and ecology from the previous summer as the basis of their predictions for the current summer. Two methods are used to read in this lake data, depending on whether predictions are being generated for the historic period or the future. If the past period, then a pre-processed set of lake observations can be read in directly, with no missing years. If for a future period, then daily data need to be read in, and the processing is done here to aggregate this to seasonal.

The future option will work on historic data, as long as there are no missing years in the daily data referred to (e.g. it won't work for 2000, as 1999 was missing most data, or for 1999-2012 (incl.), when there was no NIVA colour data (during this period, MOVAR colour data was used to patch the NIVA data when generating the historic time series).

## Set up

Set the target year the forecast will be produced for, and whether to:
1) read in pre-processed seasonally-aggregated lake water chemistry/ecology observations. Preprocessed seasonal observations are currently available for the period 1981-2019. This should be chosen if you're actually interested in the forecast for the historic period, as the historic observations have been pre-processed.

2) calculate seasonally-aggregated observed lake chem/ecol from daily data here.  This should be chosen for operational forecasting of future seasons, where the year will be missing from the historic pre-processed data.

To test the code for processing the daily data (which will be needed for operational predictions), you can set use_preprocessed_historic_obs to False, but then **set the target year to e.g. after 2012**, as there are missing years in the lake water chem/ecol data, which means the BN 'predict' function doesn't work (these gaps are patched in various ways in the pre-processed observed series). Also a different set of colour data (MOVAR data, not NIVA data) were used in the period 1999-2012 than are in the daily data read in, so the predictions will differ from those used in the hindcast stats and plots.

In [2]:
# User choices
seasonal_obs_fpath = r'../Data/BN_TrainingData/TrainingData_GaussianBN_era5_1981-2019.csv'
daily_obs_fpath = r'../../Data/Observed_Chem_Ecol/Van2_Vanemfjorden_chem_obs.csv'

# ------------------------------------------------------------------------------
# Read in historic lake chem and ecol data from the previous summer

# 1) Use preprocessed, seasonally-aggregated, cleaned & patched observations up to 2019.
seasonal_obs_df = pd.read_csv(seasonal_obs_fpath, index_col=0, parse_dates=True)

# # Just pick the data for the year before the target year, and for columns of interest
# previous_summer_waterquality_df = seasonal_obs_df.loc['%s-01-01'%(target_yr-1): '%s-12-31'%(target_yr-1),['colour', 'TP', 'chla']]
# previous_summer_waterquality_df.index = previous_summer_waterquality_df.index.year
# previous_summer_waterquality_df.columns = ['colour','TP','chla']
# display(previous_summer_waterquality_df)


In [3]:
# 2) Read in daily data from post 2019 and seasonally-aggregate
now = dt.datetime.now()
current_yr = now.year

# Read in chl-a, TP and colour daily data
daily_obs_fpath = r'../../Data/Observed_Chem_Ecol/Van2_Vanemfjorden_chem_obs.csv'
daily_df = pd.read_csv(daily_obs_fpath, index_col=0, parse_dates=True, dayfirst=True)

# Read in daily chl-a biovolume data
daily_cyano_fpath = r'../../Data/Observed_Chem_Ecol/Van2_Vanemfjorden_Cyanobacteria.csv'
cyano_df = pd.read_csv(daily_cyano_fpath, index_col=0, parse_dates=True, dayfirst=True)
# convert units to mm3/l (mg/l if assume density is same as water)
cyano_df['cyano'] = cyano_df['Cyano_biovol_mm3_per_m3']/1000.
cyano_df.drop(['Cyano_biovol_mm3_per_m3'], axis=1, inplace=True)

# Join data and truncate to time period of interest & tidy
daily_df = daily_df.join(cyano_df)
daily_df = daily_df.asfreq('D').loc['%s-01-01'%(2019): '%s-12-31'%(current_yr-1)]

var_rename_dict = {'Biovolume_mm3_per_l':'Biovolume',
                   'chl-a':'chla', 'cyano':'cyano'}
daily_df.rename(var_rename_dict, axis=1, inplace=True)

# Aggregate to seasonal frequency (maximum of cyano, mean of everything else)
season_df = bn.daily_to_summer_season(daily_df)
display(season_df)

,colour,TP,chla,cyano
year,,,,
2019,47.000000,25.800000,14.983333,0.592
2020,49.333333,31.833333,11.775000,0.167


## Produce the forecast

In [46]:
# 1) For lake TP, cyano and colour
# Use the pre-fitted BN, excluding meteorological nodes when setting the 'evidence'

# Fitted bnlearn object
rfile_fpath = "../Data/RData/Vansjo_fitted_GaussianBN_era5_1981-2019.rds"

# Make predictions for target season
forecast_df = bn.bayes_net_predict_operational(rfile_fpath,
                          float(target_yr),
                          float(previous_summer_waterquality_df['chla']),
                          float(previous_summer_waterquality_df['colour']),
                          float(previous_summer_waterquality_df['TP'])
                         )

# Re-order cols
forecast_df = forecast_df[['year', 'node', 'threshold','prob_below_threshold', 
                           'prob_above_threshold', 'expected_value', 'WFD_class']]

# ----------------------------------------------------------------------------------
# 2) For chl-a
# Estimate using naive seasonal forecast (i.e. lake observations from the previous summer (May-Oct average))

chla_forecast = previous_summer_waterquality_df.loc[target_yr-1, 'chla']

# Corresponding expected WFD class (just split into <20: Moderate or better, or >20: Poor or worse)
chla_class = bn.discretize([20.0], chla_forecast)

# Add predictions to the df containing all forecasts
forecast_df.loc[len(forecast_df)+1] = [target_yr,'chla', 20.0, np.NaN, np.NaN, chla_forecast, chla_class]

# ----------------------------------------------------------------------------------
# Tidy and display
forecast_df = forecast_df.set_index('node')
forecast_df

,year,threshold,prob_below_threshold,prob_above_threshold,expected_value,WFD_class
node,,,,,,
colour,2001,48.0,0.74,0.26,41.800000,0
cyano,2001,1.0,0.36,0.64,1.460000,1
TP,2001,29.5,0.37,0.63,30.600000,1
chla,2001,20.0,NaN,NaN,22.166667,1


## Add skill information

Read in pre-calculated skill information for the historic period (1981-2019), to accompany the forecast

In [35]:
# Historic goodness-of-fit stats for variables predicted using bayesian network (derived from cross validation)

cv_stats_fpath = r'../Data/CrossValidation/Stats/CV_results_predictableNodes_era5-vs-nomet.csv'
cv_stats = pd.read_csv(cv_stats_fpath)

# Drop stats for chla (not relevant as BN not used), and for era5-driven met (also not used)
cv_stats = cv_stats.loc[cv_stats['Variable'] != 'chla']
cv_stats = cv_stats.loc[cv_stats['met_included'] == 'nomet']
cv_stats = cv_stats.drop('met_included', axis=1).set_index('Variable')
display(cv_stats)

,mean_CC,mean_rmse,mean_class_error,mean_mcc,mean_ROC_AUC
Variable,,,,,
TP,0.577181,3.915841,0.325641,0.342134,0.669444
colour,0.818836,9.360379,0.230769,0.471405,0.730769
cyano,0.674139,0.950204,0.125000,0.777620,0.884615


In [36]:
# Historic gof stats from naive forecaster
naive_stats_fpath = r'../Hindcast_stats_plots/GoF_sim_vs_obs_1981-2019.csv'
naive_stats = pd.read_csv(naive_stats_fpath)

# Just pick for chla, naive forecast
naive_stats_chla = naive_stats.loc[naive_stats['var']=='chla'].loc[naive_stats['model']=='sim_naive'].set_index('var')
naive_stats_chla

,model,pearsons_cc,spearman_cc,mae,rmse,bias,mape,mathews_cc,roc_auc_score,classification_error
var,,,,,,,,,,
chla,sim_naive,0.648,0.631,3.614,4.6,0.058,5.054,0.706,0.853,0.108


In [37]:
# Rename and join the two dfs, dropping some skill info

cv_stats = cv_stats.rename({'mean_CC':'Pearsons r',
                            'mean_rmse':'RMSE',
                            'mean_class_error': 'Classification error',
                            'mean_mcc': 'Matthews correlation coefficient',
                            'mean_ROC_AUC': 'Area under ROC curve'}, axis=1)

naive_stats_chla = naive_stats_chla[['pearsons_cc','rmse','mathews_cc','roc_auc_score','classification_error']]
naive_stats_chla = naive_stats_chla.rename({'pearsons_cc':'Pearsons r',
                                           'rmse':'RMSE',
                                           'mathews_cc':'Matthews correlation coefficient',
                                           'roc_auc_score':'Area under ROC curve',
                                           'classification_error':'Classification error'}, axis=1)

cv_stats = cv_stats.append(naive_stats_chla, sort=False)

cv_stats

,Pearsons r,RMSE,Classification error,Matthews correlation coefficient,Area under ROC curve
TP,0.577181,3.915841,0.325641,0.342134,0.669444
colour,0.818836,9.360379,0.230769,0.471405,0.730769
cyano,0.674139,0.950204,0.125000,0.777620,0.884615
chla,0.648000,4.600000,0.108000,0.706000,0.853000


# Populate the tables in the lake forecast app/pdf

The forecast has four tables. Those for TP, cyano and colour are almost exactly the same (apart from the first column label is just 'Class' for colour, not 'WFD class'). chl-a is almost the same as the others, but 'Probability of class' column is replaced by 'Predicted class'.

The information in the layout design word doc can then hopefully be populated from the two dataframes forecast_df and cv_stats. Hopefully it's all fairly self-explanatory, but in case:

From forecast_df:

- The 0 or 1 in WFD_class needs converting to the text description of the class used in the actual forecast. 0 is always the class corresponding to lower concentrations (better water quality). e.g.

        units_dict = {'TP': ug/l, etc.}

        WFD_class_dict = {'TP': {0:'Upper Moderate or better (<%s %s)' %(threshold, units_dict['TP']),
                                 1: 'Lower Moderate or worse'},
                          'chla': etc.}

- Probability of class: prob_below_threshold (this is the probability of being in the better WFD class) and prob_above_threshold (the worse of the two classes). This then needs discretizing into four classes, which will allow for auto-generation of the text (Very low, low, medium, high), and the associated colour. See K:\Prosjekter\Ferskvann\O-17323 WATExR\05_IntegratedTool\LayoutDesign\Draft2\GuidanceDoc_InterpretingLakeForecast.docx for rules.

- Forecasted value: expected_value

Then the stats are hopefully pretty obvious, except:
- MCC: again, this is discretized into four classes, see the guidance doc for the rules, accompanying text label, and colour scheme.
- An overall confidence level is then calculated, combining the MCC class and how spread the forecast is between the two classes (see final bullet points on the first page of the interpreting lake forecasts guidance doc, under 'Forecast summary'. Feel free to alter too, as long as the guidance doc is altered too).

The text forecast summary can then be auto-generated, something like:
        '{var} is expected to be {WFD_class}. Confidence level: {overall_confidence_level}'
        
The suggested lake water quality layout includes links to time series plots summarising historic skill for each water quality variable. These are here:
WATExR/Norway_Morsa/BayesianNetwork/Hindcast_stats_plots/Timeseries_gof
